## Qubits and Gates

In this guide, we will explore high-level components: qubits, qubit pairs, and circuit-level operations.
These high-level components can be used to write QUA programs at a circuit level.
At the end of this tutorial, it is possible to create the following circuit-level program:

```python
with program() as prog:
    x180(q1)
    clifford(q1, 4)
    qubit_state = measure(q1)
```

The tutorial is structured as follows:
- Define a QUAM qubit class
- Instantiate a QUAM state using the newly-defined qubits
- Registering a qubit pulse macro: `x180`
- Creating custom qubit macros
  - `measure` macro
  - `clifford` macro

## Defining a Qubit-Level Component

First we will define 

In [1]:
from typing import Dict, Optional
import numpy as np
from dataclasses import field

from quam.components.ports import FEMPortsContainer
from quam.core import QuamRoot, quam_dataclass
from quam.components.quantum_components import Qubit, QubitPair
from quam.components import MWChannel, InOutMWChannel, pulses


@quam_dataclass
class Transmon(Qubit):
    xy: MWChannel
    resonator: Optional[InOutMWChannel] = None


@quam_dataclass
class Quam(QuamRoot):
    ports: FEMPortsContainer
    qubits: Dict[str, Qubit] = field(default_factory=dict)
    qubit_pairs: Dict[str, QubitPair] = field(default_factory=dict)

2025-03-28 14:27:57,086 - qm - INFO     - Starting session: b6c337e3-4353-47ff-82bb-3bb999b0a9e1


## Instantiate QUAM

In [2]:
machine = Quam(ports=FEMPortsContainer())

## Add qubits
q1_xy_port = machine.ports.get_mw_output("con1", 1, 1, create=True)
q1_resonator_out_port = machine.ports.get_mw_output("con1", 1, 2, create=True)
q1_resonator_in_port = machine.ports.get_mw_input("con1", 1, 2, create=True)
q1 = machine.qubits["q1"] = Transmon(
    id="q1",
    xy=MWChannel(intermediate_frequency=100e6, opx_output=q1_xy_port.get_reference()),
    resonator=InOutMWChannel(
        intermediate_frequency=100e6,
        opx_output=q1_resonator_out_port.get_reference(),
        opx_input=q1_resonator_in_port.get_reference(),
    ),
)

q2_xy_mw_output = machine.ports.get_mw_output("con1", 1, 2, create=True)
q2 = machine.qubits["q2"] = Transmon(
    id="q2",
    xy=MWChannel(
        intermediate_frequency=100e6, opx_output=q2_xy_mw_output.get_reference()
    ),
)


## Add qubit pairs
machine.qubit_pairs["q1@q2"] = QubitPair(
    qubit_control=q1.get_reference(), qubit_target=q2.get_reference()
)

Access qubit pair

In [3]:
q1

Transmon(id='q1', macros={}, xy=MWChannel(operations={}, id=None, digital_outputs={}, sticky=None, intermediate_frequency=100000000.0, thread=None, core=None, LO_frequency=5000000000.0, RF_frequency=5100000000.0, opx_output=MWFEMAnalogOutputPort(controller_id='con1', fem_id=1, port_id=1, band=1, upconverter_frequency=5000000000.0, upconverters=None, delay=0, shareable=False, sampling_rate=1000000000.0, full_scale_power_dbm=-11), upconverter=1), resonator=InOutMWChannel(operations={}, id=None, digital_outputs={}, sticky=None, intermediate_frequency=100000000.0, thread=None, core=None, opx_input=MWFEMAnalogInputPort(controller_id='con1', fem_id=1, port_id=2, band=1, downconverter_frequency=5000000000.0, gain_db=None, sampling_rate=1000000000.0, shareable=False), time_of_flight=24, smearing=0, LO_frequency=5000000000.0, RF_frequency=5100000000.0, opx_output=MWFEMAnalogOutputPort(controller_id='con1', fem_id=1, port_id=2, band=1, upconverter_frequency=5000000000.0, upconverters=None, delay

In [4]:
q1 @ q2

QubitPair(id='q1@q2', macros={}, qubit_control=Transmon(id='q1', macros={}, xy=MWChannel(operations={}, id=None, digital_outputs={}, sticky=None, intermediate_frequency=100000000.0, thread=None, core=None, LO_frequency=5000000000.0, RF_frequency=5100000000.0, opx_output=MWFEMAnalogOutputPort(controller_id='con1', fem_id=1, port_id=1, band=1, upconverter_frequency=5000000000.0, upconverters=None, delay=0, shareable=False, sampling_rate=1000000000.0, full_scale_power_dbm=-11), upconverter=1), resonator=InOutMWChannel(operations={}, id=None, digital_outputs={}, sticky=None, intermediate_frequency=100000000.0, thread=None, core=None, opx_input=MWFEMAnalogInputPort(controller_id='con1', fem_id=1, port_id=2, band=1, downconverter_frequency=5000000000.0, gain_db=None, sampling_rate=1000000000.0, shareable=False), time_of_flight=24, smearing=0, LO_frequency=5000000000.0, RF_frequency=5100000000.0, opx_output=MWFEMAnalogOutputPort(controller_id='con1', fem_id=1, port_id=2, band=1, upconverter_f

Add pulse and pulse macro to qubit

## Applying a qubit macro

In [5]:
from quam.components.macro import PulseMacro

q1.xy.operations["x180"] = pulses.SquarePulse(amplitude=0.2, length=100)
q1.macros["x180"] = PulseMacro(pulse=q1.xy.operations["x180"].get_reference())

In [6]:
from qm import generate_qua_script, qua

with qua.program() as prog:
    q1.apply("x180")

print(generate_qua_script(prog))


# Single QUA script generated at 2025-03-28 14:27:57.178879
# QUA library version: 1.2.2a4

from qm import CompilerOptionArguments
from qm.qua import *

with program() as prog:
    play("x180", "q1.xy")


config = None

loaded_config = None




## Creating operations

In [7]:
from quam.core.operation import OperationsRegistry

operations_registry = OperationsRegistry()


@operations_registry.register_operation
def x180(qubit: Qubit, **kwargs):
    pass

In [8]:
with qua.program() as prog:
    x180(q1)

print(generate_qua_script(prog))


# Single QUA script generated at 2025-03-28 14:27:57.191088
# QUA library version: 1.2.2a4

from qm import CompilerOptionArguments
from qm.qua import *

with program() as prog:
    play("x180", "q1.xy")


config = None

loaded_config = None




## Creating custom macros

### Measure macro

In [9]:
from quam.components.macro import QubitMacro
from quam.core import quam_dataclass

# If using qm-qua >=1.2.2, you can replace this with `from qm.qua.type_hints import QuaVariable; QuaVariable[bool]`
from quam.utils.qua_types import QuaVariableBool


@quam_dataclass
class MeasureMacro(QubitMacro):
    def apply(self, **kwargs) -> QuaVariableBool:
        I, Q = self.qubit.resonator.measure("readout")
        qubit_state = qua.declare(bool)
        qua.assign(
            qubit_state, I > self.qubit.resonator.operations["readout"].threshold
        )
        # is_excited = qua.declare(bool, I > self.qubit.xy.operations["readout"].threshold)
        return qubit_state

In [10]:
q1.resonator.operations["readout"] = pulses.SquareReadoutPulse(
    length=1000, amplitude=0.1, threshold=0.215
)
q1.macros["measure"] = MeasureMacro()

In [11]:
with qua.program() as prog:
    qubit_state = q1.apply("measure")

print(generate_qua_script(prog))



# Single QUA script generated at 2025-03-28 14:27:57.212917
# QUA library version: 1.2.2a4

from qm import CompilerOptionArguments
from qm.qua import *

with program() as prog:
    v1 = declare(fixed, )
    v2 = declare(fixed, )
    v3 = declare(bool, )
    measure("readout", "q1.resonator", dual_demod.full("iw1", "iw2", v1), dual_demod.full("iw3", "iw1", v2))
    assign(v3, (v1>0.215))


config = None

loaded_config = None




In [12]:
@operations_registry.register_operation
def measure(qubit: Qubit, **kwargs) -> QuaVariableBool:
    pass

In [13]:
with qua.program() as prog:
    qubit_state = measure(q1)

print(generate_qua_script(prog))



# Single QUA script generated at 2025-03-28 14:27:57.225001
# QUA library version: 1.2.2a4

from qm import CompilerOptionArguments
from qm.qua import *

with program() as prog:
    v1 = declare(fixed, )
    v2 = declare(fixed, )
    v3 = declare(bool, )
    measure("readout", "q1.resonator", dual_demod.full("iw1", "iw2", v1), dual_demod.full("iw3", "iw1", v2))
    assign(v3, (v1>0.215))


config = None

loaded_config = None




### Clifford macro

In [14]:


@quam_dataclass
class CliffordMacro(QubitMacro):
    def apply(self, clifford_idx: int, **kwargs):
        with qua.switch_(clifford_idx, unsafe=True):
            with qua.case_(0):
                wait_duration = self.qubit.xy.operations["x180"].length // 4
                self.qubit.xy.wait(wait_duration)
            with qua.case_(1):
                self.qubit.xy.play("x180")
            with qua.case_(2):
                self.qubit.xy.play("y180")
            with qua.case_(3):
                self.qubit.xy.play("y180")
                self.qubit.xy.play("x180")
            with qua.case_(4):
                self.qubit.xy.play("x90")
                self.qubit.xy.play("y90")
            ...

In [15]:
q1.xy.operations["x90"] = pulses.SquarePulse(amplitude=0.1, length=100, axis_angle=0)
q1.xy.operations["x180"] = pulses.SquarePulse(amplitude=0.2, length=100, axis_angle=0)
q1.xy.operations["y90"] = pulses.SquarePulse(
    amplitude=0.1, length=100, axis_angle=np.pi / 2
)
q1.xy.operations["y180"] = pulses.SquarePulse(
    amplitude=0.2, length=100, axis_angle=np.pi / 2
)

q1.macros["clifford"] = CliffordMacro()

In [16]:
with qua.program() as prog:
    clifford_idx = qua.declare(int, 0)
    q1.apply("clifford", clifford_idx)

print(generate_qua_script(prog))


# Single QUA script generated at 2025-03-28 14:27:57.253948
# QUA library version: 1.2.2a4

from qm import CompilerOptionArguments
from qm.qua import *

with program() as prog:
    v1 = declare(int, value=0)
    with if_((v1==0), unsafe=True):
        wait(25, "q1.xy")
    with elif_((v1==1)):
        play("x180", "q1.xy")
    with elif_((v1==2)):
        play("y180", "q1.xy")
    with elif_((v1==3)):
        play("y180", "q1.xy")
        play("x180", "q1.xy")
    with elif_((v1==4)):
        play("x90", "q1.xy")
        play("y90", "q1.xy")


config = None

loaded_config = None




In [17]:
@operations_registry.register_operation
def clifford(qubit: Qubit, clifford_idx: int, **kwargs):
    pass

In [18]:
with qua.program() as prog:
    clifford_idx = qua.declare(int, 0)
    clifford(q1, clifford_idx)

print(generate_qua_script(prog))


# Single QUA script generated at 2025-03-28 14:27:57.277819
# QUA library version: 1.2.2a4

from qm import CompilerOptionArguments
from qm.qua import *

with program() as prog:
    v1 = declare(int, value=0)
    with if_((v1==0), unsafe=True):
        wait(25, "q1.xy")
    with elif_((v1==1)):
        play("x180", "q1.xy")
    with elif_((v1==2)):
        play("y180", "q1.xy")
    with elif_((v1==3)):
        play("y180", "q1.xy")
        play("x180", "q1.xy")
    with elif_((v1==4)):
        play("x90", "q1.xy")
        play("y90", "q1.xy")


config = None

loaded_config = None


